In [11]:
import ast
import pandas as pd
import numpy as np

nodes_df = pd.read_csv('nodes.csv' , header = None , names=['node_attribute'], index_col=0)
edges_df = pd.read_csv('edges.csv')


nodes_list = [(index, ast.literal_eval(row.node_attribute))  for index, row in nodes_df.iterrows()]
edges_list = [(row.From, row.To) for index, row in edges_df.iterrows()]

number_of_days = 1448
nodes_list_vec = []
for index,dict_node in nodes_list:
    vec = np.zeros(number_of_days,dtype=np.float32)
    for day, count in dict_node.items():
        vec[day] = count

    nodes_list_vec.append((index, vec))

nodes_list_vec[:10]

[(0, array([0., 0., 0., ..., 0., 0., 0.], shape=(1448,), dtype=float32)),
 (1, array([0., 0., 0., ..., 0., 0., 0.], shape=(1448,), dtype=float32)),
 (2, array([0., 0., 0., ..., 0., 0., 0.], shape=(1448,), dtype=float32)),
 (3, array([0., 0., 0., ..., 0., 0., 0.], shape=(1448,), dtype=float32)),
 (4, array([0., 0., 0., ..., 0., 0., 0.], shape=(1448,), dtype=float32)),
 (5, array([0., 0., 0., ..., 0., 0., 0.], shape=(1448,), dtype=float32)),
 (6, array([0., 0., 0., ..., 0., 0., 0.], shape=(1448,), dtype=float32)),
 (7, array([0., 0., 0., ..., 0., 0., 0.], shape=(1448,), dtype=float32)),
 (8, array([0., 0., 0., ..., 0., 0., 0.], shape=(1448,), dtype=float32)),
 (9, array([0., 0., 0., ..., 0., 0., 0.], shape=(1448,), dtype=float32))]

In [14]:
import networkx as nx
import matplotlib.pyplot as plt

# Assuming you've already created your graph as shown
Graph = nx.DiGraph()
for node_id, node_attr_vec in nodes_list_vec:
    Graph.add_node(node_id, x = node_attr_vec)
Graph.add_edges_from(edges_list)

print(f"Number of nodes: {nx.number_of_nodes(Graph)}")
print(f"Number of edges: {nx.number_of_edges(Graph)}")

Number of nodes: 6600
Number of edges: 50897


In [15]:
import torch
from torch_geometric.utils import from_networkx 

data = from_networkx(Graph)

print(data)

Data(x=[6600, 1448], edge_index=[2, 50897])


/home/barsat/Documents/Social-Engineering-Attack-Prevention-in-Coorporate-/GNNProject/venv/lib/python3.12/site-packages/torch_geometric/utils/convert.py:278: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  data_dict[key] = torch.as_tensor(value)
